In [28]:
from definitions import data_source_path, corr_country_names, sign_level, fake_num, show_plots, \
    country_col, year_col, quarter_col, month_col, date_col, incl_countries, show_results
from helper_functions_general import read_data, select_country_year_measure, month_name_to_num, rename_order_scale, \
    downsample_month_to_quarter, quarter_to_month, upsample_quarter_to_month, get_timeframe_col, get_trans, \
    get_data_path, interpolate_series, get_table_path, get_impl_date

import matplotlib.pyplot as plt
import pandas as pd

from scipy.stats import ttest_1samp, shapiro

In [29]:
timeframe = 'm'
model = 'arco'

country_weight = {'switzerland': 0.9,
                  'ireland': 1.0,
                  'united_kingdom': 0.8,
                  'france': 1.0,
                  'portugal': 0.65
                  }

In [38]:
treatment_country = 'portugal'

# act_pred_log_diff

In [39]:
var_title = 'act_pred_log_diff'

tables_path_res = get_table_path(timeframe=timeframe, folder='results', country=treatment_country)
var_name = f'{model}_{treatment_country}_{timeframe}_{var_title}'

act_pred_log_diff = read_data(source_path=tables_path_res, file_name=var_name)
act_pred_log_diff = act_pred_log_diff.set_index(date_col)
act_pred_log_diff

act_pred_log_diff_pre = act_pred_log_diff[act_pred_log_diff.index < get_impl_date(treatment_country=treatment_country)]
act_pred_log_diff_pre

act_pred_log_diff_post = act_pred_log_diff[act_pred_log_diff.index >= get_impl_date(treatment_country=treatment_country)]
act_pred_log_diff_post

# plt.plot(act_pred_log_diff[['act', 'pred']])
# plt.plot(act_pred_log_diff['error'])

,act,pred,error
date,,,
2015-01-01,0.149161,0.019141,-0.130019
2015-02-01,0.128294,0.011123,-0.117171
2015-03-01,0.050892,-0.004544,-0.055436
2015-04-01,0.114095,0.016165,-0.097930
2015-05-01,0.001809,-0.003894,-0.005702
2015-06-01,0.065981,0.009623,-0.056358
2015-07-01,0.060651,0.021785,-0.038866
2015-08-01,0.043263,0.002597,-0.040667
2015-09-01,0.030482,-0.003448,-0.033930


In [40]:
# pre
print('pre-treatment data')
df_pre = act_pred_log_diff_pre
train_weight = int(country_weight[treatment_country] * len(df_pre))
df_pre = df_pre[:train_weight]

print(f"mean: {df_pre['error'].mean()}")
print(f"std:  {df_pre['error'].std()}")

shap = shapiro(df_pre['error'])
if shap[1] > sign_level:
    print(f'Errors are normally distributed (p-value={round(shap[1],3)})')
else:
    print(f'Errors NOT normally distributed (p-value={round(shap[1],3)})')

ttest_res = ttest_1samp(df_pre['error'], popmean=0)
if ttest_res[1] < sign_level:
    print(f'Significant (p-value={round(ttest_res[1],3)})')
else:
    print(f'NOT significant (p-value={round(ttest_res[1],3)})')

pre-treatment data
mean: -0.0016182442012250592
std:  0.08237742113832351
Errors are normally distributed (p-value=0.129)
NOT significant (p-value=0.844)


In [41]:
# post
df_post = act_pred_log_diff_post
print('post-treatment data')

print(f"sum: {df_post['error'].sum()}")
print(f"mean: {df_post['error'].mean()}")
print(f"std:  {df_post['error'].std()}")

ttest_res = ttest_1samp(df_post['error'], popmean=0)
if ttest_res[1] < sign_level:
    print(f'Significant (p-value={round(ttest_res[1],3)})')
else:
    print(f'NOT significant (p-value={round(ttest_res[1],3)})')

post-treatment data
sum: -0.6017875057753153
mean: -0.010375646651298543
std:  0.06293352456699923
NOT significant (p-value=0.214)


# act_pred

In [42]:
var_title = 'act_pred'

tables_path_res = get_table_path(timeframe=timeframe, folder='results', country=treatment_country)
var_name = f'{model}_{treatment_country}_{timeframe}_{var_title}'

act_pred = read_data(source_path=tables_path_res, file_name=var_name)
act_pred = act_pred.set_index(date_col)
act_pred

act_pred_pre = act_pred[act_pred.index < get_impl_date(treatment_country=treatment_country)]
act_pred_pre

act_pred_post = act_pred[act_pred.index >= get_impl_date(treatment_country=treatment_country)]
act_pred_post

# plt.plot(act_pred_log_diff[['act', 'pred']])
# plt.plot(act_pred_log_diff['error'])

,act,pred,error
date,,,
2015-01-01,6.634756e+09,6.754008e+09,-1.192523e+08
2015-02-01,6.121384e+09,6.322517e+09,-2.011338e+08
2015-03-01,5.870068e+09,5.920246e+09,-5.017776e+07
2015-04-01,5.416318e+09,5.579873e+09,-1.635546e+08
2015-05-01,5.015950e+09,5.350654e+09,-3.347041e+08
2015-06-01,5.431635e+09,5.018041e+09,4.135934e+08
2015-07-01,5.476822e+09,5.252362e+09,2.244607e+08
2015-08-01,5.115376e+09,4.669218e+09,4.461577e+08
2015-09-01,5.337175e+09,4.849361e+09,4.878143e+08


In [43]:
# pre
print('pre-treatment data')
df_pre = act_pred_log_diff_pre
train_weight = int(country_weight[treatment_country] * len(df_pre))
df_pre = df_pre[:train_weight]
print(f"mean: {df_pre['error'].mean()}")
print(f"std:  {df_pre['error'].std()}")

shap = shapiro(df_pre['error'])
if shap[1] > sign_level:
    print(f'Errors are normally distributed (p-value={round(shap[1],3)})')
else:
    print(f'Errors NOT normally distributed (p-value={round(shap[1],3)})')


ttest_res = ttest_1samp(df_pre['error'], popmean=0)
if ttest_res[1] < sign_level:
    print(f'Significant (p-value={round(ttest_res[1],3)})')
else:
    print(f'NOT significant (p-value={round(ttest_res[1],3)})')

pre-treatment data
mean: -0.0016182442012250592
std:  0.08237742113832351
Errors are normally distributed (p-value=0.129)
NOT significant (p-value=0.844)


In [44]:
# post
df_post = act_pred_post
print('post-treatment data')

print(f"sum: {df_post['error'].sum()}")
print(f"mean: {df_post['error'].mean()}")
print(f"std:  {df_post['error'].std()}")

ttest_res = ttest_1samp(df_post['error'], popmean=0)
if ttest_res[1] < sign_level:
    print(f'Significant (p-value={round(ttest_res[1],3)})')
else:
    print(f'NOT significant (p-value={round(ttest_res[1],3)})')

post-treatment data
sum: 15945921686.142529
mean: 274929684.24383676
std:  453452915.36873794
Significant (p-value=0.0)


In [45]:
act_pred_post['error'].sum()

15945921686.142529